In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import time

In [0]:
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE default")

print("="*70)
print("ENVIRONMENT SETUP")
print("="*70)
print(f"✓ Current Catalog: {spark.sql('SELECT current_catalog()').collect()[0][0]}")
print(f"✓ Current Database: {spark.sql('SELECT current_database()').collect()[0][0]}")
print(f"✓ Spark version: {spark.version}")
print("="*70)

ENVIRONMENT SETUP
✓ Current Catalog: hive_metastore
✓ Current Database: default
✓ Spark version: 4.0.0


In [0]:
# ============================================
# Load All Data
# ============================================

start_time = time.time()

# Read all 2023 data at once
df_raw = spark.read.parquet("/mnt/taxi-data/yellow_tripdata_202*.parquet")

load_time = time.time() - start_time
total_rows = df_raw.count()

print("="*60)
print("DATA LOADING STATISTICS")
print("="*60)
print(f"Total Rows: {total_rows:,}")
print(f"Total Columns: {len(df_raw.columns)}")
print(f"Load Time: {load_time:.2f} seconds")
print("="*60)

DATA LOADING STATISTICS
Total Rows: 65,253,104
Total Columns: 19
Load Time: 10.03 seconds


In [0]:
# ============================================
# Data Cleaning Function
# ============================================

def clean_taxi_data(df):
    """
    Clean NYC taxi data with business logic
    DS FEATURE: Distributed data transformation across cluster
    """
    
    print("\nApplying data cleaning rules...")
    
    df_clean = df.filter(
        # Trip validation
        (col("trip_distance") > 0) & (col("trip_distance") < 100) &
        
        # Fare validation
        (col("fare_amount") > 0) & (col("fare_amount") < 500) &
        
        # Passenger validation
        (col("passenger_count") > 0) & (col("passenger_count") <= 6) &
        
        # Amount validation
        (col("total_amount") > 0) & (col("total_amount") < 500) &
        
        # Datetime validation
        col("tpep_pickup_datetime").isNotNull() &
        col("tpep_dropoff_datetime").isNotNull() &
        (col("tpep_pickup_datetime") < col("tpep_dropoff_datetime"))
    )
    
    return df_clean

# Apply cleaning
df_clean = clean_taxi_data(df_raw)
clean_count = df_clean.count()

print(f"\nOriginal rows: {total_rows:,}")
print(f"Clean rows: {clean_count:,}")
print(f"Removed: {total_rows - clean_count:,} ({((total_rows - clean_count)/total_rows)*100:.1f}%)")

# Show sample of clean data
print("\nSample clean data:")
df_clean.select("tpep_pickup_datetime", "trip_distance", "fare_amount", "passenger_count").show(10)


Applying data cleaning rules...

Original rows: 65,253,104
Clean rows: 53,338,965
Removed: 11,914,139 (18.3%)

Sample clean data:
+--------------------+-------------+-----------+---------------+
|tpep_pickup_datetime|trip_distance|fare_amount|passenger_count|
+--------------------+-------------+-----------+---------------+
| 2025-05-01 00:07:06|          3.7|       18.4|              1|
| 2025-05-01 00:07:44|         1.03|        8.6|              1|
| 2025-05-01 00:15:56|         1.57|       10.0|              1|
| 2025-05-01 00:00:09|         9.48|       40.8|              1|
| 2025-05-01 00:45:07|          1.8|       10.0|              1|
| 2025-05-01 00:09:24|         5.11|       22.6|              1|
| 2025-04-30 23:50:34|         0.99|        7.9|              2|
| 2025-05-01 00:04:45|         0.47|        5.1|              1|
| 2025-05-01 00:09:36|         1.33|        8.6|              1|
| 2025-05-01 00:04:30|         4.71|       21.2|              1|
+--------------------+--

In [0]:
# ============================================
# Feature Engineering
# DS FEATURE: Distributed feature computation
# ============================================

print("\nCreating features from 2024-2025 data...")

df_features = df_clean.withColumn(
    "trip_duration_minutes",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
).withColumn(
    "hour_of_day",
    hour("tpep_pickup_datetime")
).withColumn(
    "day_of_week",
    dayofweek("tpep_pickup_datetime")  # 1=Sunday, 7=Saturday
).withColumn(
    "month",
    month("tpep_pickup_datetime")
).withColumn(
    "year",
    year("tpep_pickup_datetime")
).withColumn(
    "is_weekend",
    when(dayofweek("tpep_pickup_datetime").isin([1, 7]), 1).otherwise(0)
).withColumn(
    "speed_mph",
    col("trip_distance") / (col("trip_duration_minutes") / 60)
).withColumn(
    "date",
    to_date("tpep_pickup_datetime")
)

# Filter target variable outliers
df_features = df_features.filter(
    (col("trip_duration_minutes") >= 1) & 
    (col("trip_duration_minutes") <= 120)  # Max 2 hours
)

# Select relevant columns
feature_columns = [
    "VendorID",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "passenger_count",
    "trip_distance",
    "PULocationID",
    "DOLocationID",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "trip_duration_minutes",
    "hour_of_day",
    "day_of_week",
    "month",
    "year",
    "is_weekend",
    "speed_mph",
    "date"
]

df_final = df_features.select(feature_columns)

print(f"\nFeatures created: {len(feature_columns)} columns")
print(f"Final dataset: {df_final.count():,} rows")

# Show feature statistics
print("\nFeature Statistics:")
df_final.select(
    "trip_duration_minutes",
    "trip_distance",
    "fare_amount",
    "speed_mph"
).describe().show()

# Show sample with new features
print("\nSample data with features:")
df_final.select(
    "date",
    "trip_distance", 
    "trip_duration_minutes", 
    "hour_of_day", 
    "day_of_week", 
    "is_weekend"
).show(10)


Creating features from 2024-2025 data...

Features created: 18 columns
Final dataset: 53,106,919 rows

Feature Statistics:
+-------+---------------------+------------------+------------------+--------------------+
|summary|trip_duration_minutes|     trip_distance|       fare_amount|           speed_mph|
+-------+---------------------+------------------+------------------+--------------------+
|  count|             53106919|          53106919|          53106919|            53106919|
|   mean|   16.600533291465936|3.4004223743425364|19.428691224196086|  10.885047366205258|
| stddev|   13.678082892443884| 4.495151794992424|17.798640050718348|   6.640058866256083|
|    min|                  1.0|              0.01|              0.01|0.005271635671401...|
|    max|                120.0|             99.86|             497.9|  2998.6813186813188|
+-------+---------------------+------------------+------------------+--------------------+


Sample data with features:
+----------+-------------+--

In [0]:
# ============================================
# DS FEATURE 1: DATA PARTITIONING
# DS FEATURE 2: ACID TRANSACTIONS
# ============================================

print("\n" + "="*70)
print("SAVING TO DELTA LAKE (2024-2025 DATA)")
print("="*70)

delta_path = "/mnt/taxi-data/delta/taxi_trips_clean"

start_time = time.time()

# Write with YEAR and MONTH partitioning for 2024-2025 data
# This creates partitions like: year=2024/month=01, year=2024/month=02, etc.
df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .option("overwriteSchema", "true") \
    .save(delta_path)

save_time = time.time() - start_time

print(f"\nData saved to Delta Lake")
print(f"Location: {delta_path}")
print(f"Save time: {save_time:.2f} seconds")
print(f"Partitioning: BY YEAR and MONTH")
print(f"Number of partitions: Multiple (year/month combinations)")
print(f"Format: Delta (ACID-compliant)")

# # Show partitioning structure
# print("\nPartition structure example:")
# print("  year=2024/")
# print("    month=01/")
# print("    month=02/")
# print("    ...")
# print("  year=2025/")
# print("    month=01/")
# print("    ...")


SAVING TO DELTA LAKE (2024-2025 DATA)

Data saved to Delta Lake
Location: /mnt/taxi-data/delta/taxi_trips_clean
Save time: 27.76 seconds
Partitioning: BY YEAR and MONTH
Number of partitions: Multiple (year/month combinations)
Format: Delta (ACID-compliant)


In [0]:
# ============================================
# Create Managed Table
# ============================================

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS taxi_trips
    USING DELTA
    LOCATION '{delta_path}'
""")

print("\n✓ Managed table created: taxi_trips")

# Verify table
table_info = spark.sql("DESCRIBE EXTENDED taxi_trips")
print("\nTable information:")
display(table_info)

# Show table statistics
print("\nTable Statistics:")
spark.sql("""
    SELECT 
        year,
        month,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration,
        ROUND(AVG(trip_distance), 2) as avg_distance
    FROM taxi_trips
    GROUP BY year, month
    ORDER BY year, month
""").show(24)


✓ Managed table created: taxi_trips

Table information:


col_name,data_type,comment
VendorID,int,null
tpep_pickup_datetime,timestamp_ntz,null
tpep_dropoff_datetime,timestamp_ntz,null
passenger_count,bigint,null
trip_distance,double,null
PULocationID,int,null
DOLocationID,int,null
fare_amount,double,null
tip_amount,double,null
total_amount,double,null



Table Statistics:
+----+-----+----------+------------+------------+
|year|month|trip_count|avg_duration|avg_distance|
+----+-----+----------+------------+------------+
|2002|   12|         5|       15.95|         3.6|
|2007|   12|         1|        17.0|         3.0|
|2008|   12|         1|       11.33|        1.62|
|2009|    1|        14|       31.13|        8.79|
|2023|   12|        10|       10.16|         2.6|
|2024|    1|   2712719|       14.93|        3.29|
|2024|    2|   2708963|       15.31|        3.22|
|2024|    3|   3023161|       16.09|        3.39|
|2024|    4|   2975153|       16.43|        3.42|
|2024|    5|   3178741|       17.45|        3.45|
|2024|    6|   2993137|        17.0|        3.45|
|2024|    7|   2670748|       16.71|        3.64|
|2024|    8|   2591178|       16.82|        3.71|
|2024|    9|   3009979|       18.18|        3.57|
|2024|   10|   3288107|       17.77|         3.5|
|2024|   11|   3132710|       17.19|        3.33|
|2024|   12|   3185044|        

In [0]:
# ============================================
# DELTA LAKE FEATURES DEMONSTRATION
# DS FEATURE: ACID TRANSACTIONS & TIME TRAVEL
# ============================================

print("\n" + "="*70)
print("DELTA LAKE FEATURES DEMONSTRATION")
print("="*70)

# Feature 1: Table History (Time Travel)
print("\n1. TABLE HISTORY (Time Travel):")
history = spark.sql("DESCRIBE HISTORY taxi_trips")
display(history.select("version", "timestamp", "operation", "operationMetrics"))

# Feature 2: Show Partitions (FIXED)
print("\n2. DATA PARTITIONING (Year/Month):")
partitions = spark.sql("SHOW PARTITIONS taxi_trips")
print(f"Total partitions: {partitions.count()}")

# Display partitions - just show the DataFrame without ordering
# The partition info is in the default column
print("\nPartition List:")
display(partitions)

# Alternative: Show partition summary by parsing the partition column
print("\nPartition Summary:")
spark.sql("""
    SELECT 
        year,
        COUNT(DISTINCT month) as months_in_year,
        MIN(month) as first_month,
        MAX(month) as last_month
    FROM taxi_trips
    GROUP BY year
    ORDER BY year
""").show()

# Feature 3: Optimize Table
print("\n3. OPTIMIZING TABLE:")
spark.sql("OPTIMIZE taxi_trips")
print("Table optimized (compacted small files)")

# Feature 4: Z-ordering for query performance
print("\n4. Z-ORDERING (Multi-dimensional Query Optimization):")
spark.sql("OPTIMIZE taxi_trips ZORDER BY (hour_of_day, day_of_week)")
print("Z-ordering applied on hour_of_day and day_of_week")
print("Benefit: Faster queries filtering by time-based features")

# Feature 5: Show partition statistics
print("\n5. PARTITION STATISTICS:")
spark.sql("""
    SELECT 
        year,
        COUNT(DISTINCT month) as months_available,
        COUNT(*) as total_trips,
        MIN(date) as earliest_date,
        MAX(date) as latest_date
    FROM taxi_trips
    GROUP BY year
    ORDER BY year
""").show()

# Feature 6: Demonstrate partition pruning benefit
print("\n6. PARTITION PRUNING DEMONSTRATION:")
print("\nQuery A: Full table scan (no partition filter)")
start_time = time.time()
count_all = spark.sql("SELECT COUNT(*) FROM taxi_trips WHERE trip_distance > 10").collect()[0][0]
time_all = time.time() - start_time
print(f"  Time: {time_all:.2f}s | Rows: {count_all:,}")

print("\nQuery B: With partition filter (year=2024, month=1)")
start_time = time.time()
count_filtered = spark.sql("SELECT COUNT(*) FROM taxi_trips WHERE year=2024 AND month=1 AND trip_distance > 10").collect()[0][0]
time_filtered = time.time() - start_time
print(f"  Time: {time_filtered:.2f}s | Rows: {count_filtered:,}")

if time_all > 0 and time_filtered > 0:
    speedup = time_all / time_filtered
    print(f"\nSpeedup with partition pruning: {speedup:.2f}x faster")
else:
    print("\nPartition pruning demonstrated (both queries completed)")




DELTA LAKE FEATURES DEMONSTRATION

1. TABLE HISTORY (Time Travel):


version,timestamp,operation,operationMetrics
7,2025-12-02T21:29:38Z,WRITE,"Map(numFiles -> 65, numRemovedFiles -> 24, numRemovedBytes -> 1509406534, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509441703)"
6,2025-12-01T04:02:05Z,OPTIMIZE,"Map(numRemovedFiles -> 61, numRemovedBytes -> 1509425486, p25FileSize -> 76466832, numDeletionVectorsRemoved -> 0, minFileSize -> 4349, numAddedFiles -> 20, maxFileSize -> 93564600, p75FileSize -> 88865064, p50FileSize -> 85191063, numAddedBytes -> 1509390317)"
5,2025-12-01T04:01:13Z,WRITE,"Map(numFiles -> 65, numRemovedFiles -> 65, numRemovedBytes -> 1509631590, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509441703)"
4,2025-12-01T03:46:28Z,WRITE,"Map(numFiles -> 65, numRemovedFiles -> 24, numRemovedBytes -> 1509048741, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509631590)"
3,2025-12-01T03:20:08Z,OPTIMIZE,"Map(numRemovedFiles -> 62, numRemovedBytes -> 1509296030, p25FileSize -> 76484634, numDeletionVectorsRemoved -> 0, minFileSize -> 4352, numAddedFiles -> 20, maxFileSize -> 93639715, p75FileSize -> 88790426, p50FileSize -> 85353644, numAddedBytes -> 1509032524)"
2,2025-12-01T02:57:06Z,WRITE,"Map(numFiles -> 66, numRemovedFiles -> 65, numRemovedBytes -> 1509441703, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509312247)"
1,2025-12-01T02:40:29Z,WRITE,"Map(numFiles -> 65, numRemovedFiles -> 65, numRemovedBytes -> 1509441703, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509441703)"
0,2025-12-01T02:38:27Z,WRITE,"Map(numFiles -> 65, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 53106919, numOutputBytes -> 1509441703)"



2. DATA PARTITIONING (Year/Month):
Total partitions: 24

Partition List:


year,month
2024,7
2024,12
2002,12
2025,5
2024,3
2025,6
2024,5
2009,1
2008,12
2024,10



Partition Summary:
+----+--------------+-----------+----------+
|year|months_in_year|first_month|last_month|
+----+--------------+-----------+----------+
|2002|             1|         12|        12|
|2007|             1|         12|        12|
|2008|             1|         12|        12|
|2009|             1|          1|         1|
|2023|             1|         12|        12|
|2024|            12|          1|        12|
|2025|             6|          1|         6|
|2026|             1|          6|         6|
+----+--------------+-----------+----------+


3. OPTIMIZING TABLE:
Table optimized (compacted small files)

4. Z-ORDERING (Multi-dimensional Query Optimization):
Z-ordering applied on hour_of_day and day_of_week
Benefit: Faster queries filtering by time-based features

5. PARTITION STATISTICS:
+----+----------------+-----------+-------------+-----------+
|year|months_available|total_trips|earliest_date|latest_date|
+----+----------------+-----------+-------------+-----------+
|20

In [0]:
# ============================================
# Data Quality Verification
# ============================================

print("\n" + "="*70)
print("DATA QUALITY VERIFICATION (2024-2025)")
print("="*70)

# Summary statistics
print("\n1. Overall Summary Statistics:")
spark.sql("""
    SELECT 
        COUNT(*) as total_trips,
        COUNT(DISTINCT year) as years_covered,
        COUNT(DISTINCT month) as months_covered,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration_min,
        ROUND(AVG(trip_distance), 2) as avg_distance_miles,
        ROUND(AVG(fare_amount), 2) as avg_fare_usd,
        MIN(date) as earliest_date,
        MAX(date) as latest_date
    FROM taxi_trips
""").show(truncate=False)

# Distribution by year
print("\n2. Trips by Year:")
spark.sql("""
    SELECT 
        year,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration,
        ROUND(AVG(fare_amount), 2) as avg_fare
    FROM taxi_trips
    GROUP BY year
    ORDER BY year
""").show()

# Distribution by month (across all years)
print("\n3. Trips by Month (All Years Combined):")
spark.sql("""
    SELECT 
        month,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration
    FROM taxi_trips
    GROUP BY month
    ORDER BY month
""").show(12)

# Distribution by hour of day
print("\n4. Trips by Hour of Day:")
spark.sql("""
    SELECT 
        hour_of_day,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration
    FROM taxi_trips
    GROUP BY hour_of_day
    ORDER BY hour_of_day
""").show(24)

# Weekend vs Weekday
print("\n5. Weekend vs Weekday Analysis:")
spark.sql("""
    SELECT 
        CASE WHEN is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END as day_type,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration,
        ROUND(AVG(fare_amount), 2) as avg_fare
    FROM taxi_trips
    GROUP BY is_weekend
    ORDER BY is_weekend
""").show()

# Speed distribution (to check for anomalies)
print("\n6. Speed Distribution Check:")
spark.sql("""
    SELECT 
        CASE 
            WHEN speed_mph < 5 THEN 'Very Slow (<5 mph)'
            WHEN speed_mph BETWEEN 5 AND 15 THEN 'Slow (5-15 mph)'
            WHEN speed_mph BETWEEN 15 AND 30 THEN 'Normal (15-30 mph)'
            WHEN speed_mph BETWEEN 30 AND 50 THEN 'Fast (30-50 mph)'
            ELSE 'Very Fast (>50 mph)'
        END as speed_category,
        COUNT(*) as trip_count,
        ROUND(AVG(trip_duration_minutes), 2) as avg_duration
    FROM taxi_trips
    GROUP BY speed_category
    ORDER BY speed_category
""").show()

# print("\n" + "="*70)
# print("✓ DATA PIPELINE COMPLETE (2024-2025)")
# print("✓ Data ready for ML training")
# print("✓ Time period covered: 2024-2025")
print(f"✓ Total clean trips: {df_final.count():,}")
# print("="*70)


DATA QUALITY VERIFICATION (2024-2025)

1. Overall Summary Statistics:
+-----------+-------------+--------------+----------------+------------------+------------+-------------+-----------+
|total_trips|years_covered|months_covered|avg_duration_min|avg_distance_miles|avg_fare_usd|earliest_date|latest_date|
+-----------+-------------+--------------+----------------+------------------+------------+-------------+-----------+
|53106919   |8            |12            |16.6            |3.4               |19.43       |2002-12-31   |2026-06-26 |
+-----------+-------------+--------------+----------------+------------------+------------+-------------+-----------+


2. Trips by Year:
+----+----------+------------+--------+
|year|trip_count|avg_duration|avg_fare|
+----+----------+------------+--------+
|2002|         5|       15.95|   18.96|
|2007|         1|        17.0|    17.0|
|2008|         1|       11.33|    11.4|
|2009|        14|       31.13|   35.97|
|2023|        10|       10.16|   14.41|